In [1]:
!cd HomeWorks/hw2

‘ЁбвҐ¬Ґ ­Ґ г¤ Ґвбп ­ ©вЁ гЄ § ­­л© Їгвм.


Датасет:
https://www.kaggle.com/c/kkbox-music-recommendation-challenge/overview

Можете мерять результат по 5-fold evaluation на train-set.

Soft dead:
25.10
Dead dead:
01.11

# Импорты

In [118]:
import pandas as pd
import xgboost as xgb
import numpy as np
from sklearn.preprocessing import OneHotEncoder
import datetime

# Данные

In [97]:
dataset_path = "dataset/"
members = pd.read_csv(dataset_path + "members.csv")
sample_submission = pd.read_csv(dataset_path + "sample_submission.csv")
song_extra_info = pd.read_csv(dataset_path + "song_extra_info.csv")
songs = pd.read_csv(dataset_path + "songs.csv")
test = pd.read_csv(dataset_path + "test.csv")
train = pd.read_csv(dataset_path + "train.csv")

In [87]:
'city	bd	gender	registered_via	registration_init_time	expiration_date'.split()

['city',
 'bd',
 'gender',
 'registered_via',
 'registration_init_time',
 'expiration_date']

In [69]:
members

,msno,city,bd,gender,registered_via,registration_init_time,expiration_date
0,XQxgAYj3klVKjR3oxPPXYYFp4soD4TuBghkhMTD4oTw=,1,0,NaN,7,20110820,20170920
1,UizsfmJb9mV54qE9hCYyU07Va97c0lCRLEQX3ae+ztM=,1,0,NaN,7,20150628,20170622
2,D8nEhsIOBSoE6VthTaqDX8U6lqjJ7dLdr72mOyLya2A=,1,0,NaN,4,20160411,20170712
3,mCuD+tZ1hERA/o5GPqk38e041J8ZsBaLcu7nGoIIvhI=,1,0,NaN,9,20150906,20150907
4,q4HRBfVSssAFS9iRfxWrohxuk9kCYMKjHOEagUMV6rQ=,1,0,NaN,4,20170126,20170613
...,...,...,...,...,...,...,...
34398,Wwd/cudKVuLJ3txRVxlg2Zaeliu+LRUfiBmfrnxhRCY=,1,0,NaN,7,20131111,20170910
34399,g3JGnJX6Hg50lFbrNWfsHwCUmApIkiv2M8sXOaeXoIQ=,4,18,male,3,20141024,20170518
34400,IMaPMJuyN+ip9Vqi+z2XuXbFAP2kbHr+EvvCNkFfj+o=,1,0,NaN,7,20130802,20170908
34401,WAnCAJjUty9Stv8yKtV7ZC7PN+ilOy5FX3aIJgGPANM=,1,0,NaN,7,20151020,20170920


In [68]:
songs

,song_id,song_length,genre_ids,artist_name,composer,lyricist,language
0,CXoTN1eb7AI+DntdU1vbcwGRV4SCIDxZu+YD8JP8r4E=,247640,465,張信哲 (Jeff Chang),董貞,何啟弘,3.0
1,o0kFgae9QtnYgRkVPqLJwa05zIhRlUjfF7O1tDw0ZDU=,197328,444,BLACKPINK,TEDDY| FUTURE BOUNCE| Bekuh BOOM,TEDDY,31.0
2,DwVvVurfpuz+XPuFvucclVQEyPqcpUkHR0ne1RQzPs0=,231781,465,SUPER JUNIOR,NaN,NaN,31.0
3,dKMBWoZyScdxSkihKG+Vf47nc18N9q4m58+b4e7dSSE=,273554,465,S.H.E,湯小康,徐世珍,3.0
4,W3bqWd3T+VeHFzHAUfARgW9AvVRaF4N5Yzm4Mr6Eo/o=,140329,726,貴族精選,Traditional,Traditional,52.0
...,...,...,...,...,...,...,...
2296315,lg6rn7eV/ZNg0+P+x77kHUL7GDMfoL4eMtXxncseLNA=,20192,958,Catherine Collard,Robert Schumann (1810-1856),NaN,-1.0
2296316,nXi1lrSJe+gLoTTNky7If0mNPrIyCQCLwagwR6XopGU=,273391,465,紀文惠 (Justine Chi),NaN,NaN,3.0
2296317,9KxSvIjbJyJzfEVWnkMbgR6dyn6d54ot0N5FKyKqii8=,445172,1609,Various Artists,NaN,NaN,52.0
2296318,UO8Y2MR2sjOn2q/Tp8/lzZTGKmLEvwZ20oWanG4XnYc=,172669,465,Peter Paul & Mary,NaN,NaN,52.0


In [14]:
train

,msno,song_id,source_system_tab,source_screen_name,source_type,target
0,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,explore,Explore,online-playlist,1
1,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,bhp/MpSNoqoxOIB+/l8WPqu6jldth4DIpCm3ayXnJqM=,my library,Local playlist more,local-playlist,1
2,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,JNWfrrC7zNN7BdMpsISKa4Mw+xVJYNnxXh3/Epw7QgY=,my library,Local playlist more,local-playlist,1
3,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,2A87tzfnJTSWqD7gIZHisolhe4DMdzkbd6LzO1KHjNs=,my library,Local playlist more,local-playlist,1
4,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,3qm6XTZ6MOCU11x8FIVbAGH5l5uMkT3/ZalWG1oo2Gc=,explore,Explore,online-playlist,1
...,...,...,...,...,...,...
7377413,6xdFzPlrasIDD95mQWXVC3Bg4ptnGYtBl4ztVEZMddU=,VJTxizih/o28kXCbtPbIyWXScoXGvxyYtl6R+0YB5JM=,my library,Local playlist more,local-playlist,1
7377414,ZxbVmt3Kh/XOH+h58c2Kdj6SjFZk+wnUO006IgWzMQE=,z1mqaU9YOX7T/PFDvUoWozdFq7rC3KwaQP7nFVprjMI=,search,Search,song,0
7377415,ZxbVmt3Kh/XOH+h58c2Kdj6SjFZk+wnUO006IgWzMQE=,750RprmFfLV0bymtDH88g24pLZGVi5VpBAI300P6UOA=,search,Search,song,1
7377416,0aH4Hd3ziPSRHClRX8rkeOEaAG5EPPkW1mKGCdXEok0=,G8wgqObgeAMER/rVCIlgcNeQ8mm0CzF/GsxiMK8TTnA=,discover,Discover Genre,online-playlist,1


Сделаем функцию, которая из датасета будет доставать фичи.

In [156]:
class FeatureExtractor:
    def __init__(self, members, sample_submission, song_extra_info, songs):
        self.members = members.copy()
        self.sample_submission = sample_submission.copy()
        self.song_extra_info = song_extra_info.copy()
        self.songs = songs.copy()
        self.members_one_hot_encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
        self.prepare_members()
        
    def prepare_members(self):
        # Убираю аутлайеры из возраста и завожу отдельную фичу для людей, которые не указали возраст
        bd = self.members.loc[:, 'bd'].copy().astype(int)
        bd_missing = (bd <= 0) | (bd > 100)
        bd.loc[bd_missing] = 0
        self.members.loc[:, 'bd'] = bd.astype(int)
        self.members.insert(3,'bd_missing', bd_missing.astype(int))
        
        # Преобразую время
        dates = ['registration_init_time', 'expiration_date']
        self.members.loc[:, dates] = self.members.loc[:, dates].applymap(self.date_transform)
        
        # Убираю NaN'ы у пола
        self.members['gender'] = self.members['gender'].fillna('miss')
        
        # Делаю one-hot фичи
        one_hot_classes = ['city', 'gender', 'registered_via']
        to_one_hot = self.members[one_hot_classes].copy()
        self.members.drop(columns=one_hot_classes, inplace = True)
        one_hot = self.members_one_hot_encoder.fit_transform(to_one_hot).astype(int)
        categories = []
        for cls, cats in zip(one_hot_classes, self.members_one_hot_encoder.categories_):
            categories.extend(map(lambda s: cls + '_' + str(s), cats))
        for i, column in enumerate(categories):
            self.members.insert(len(self.members.columns), column, one_hot[:, i])
        
    # Преобразую время в относительное
    def date_transform(self, date):
        day = date % 100
        month = (date // 100) % 100
        year = date // 10000
        delta = datetime.date(2020, 10, 28) - datetime.date(year, month, day)
        return int(delta.total_seconds() // 3600 // 24)

    def get_features(self, dataset):
        data = dataset.join(_members.set_index('msno'), on='msno').join(songs.set_index('song_id'), on='song_id')
        X, y = data.drop(columns=['msno', 'song_id', 'target']), data['target']
        return X, y

In [157]:
features = FeatureExtractor(members, sample_submission, song_extra_info, songs)

In [158]:
features.members

,msno,bd,bd_missing,registration_init_time,expiration_date,city_1,city_3,city_4,city_5,city_6,...,city_22,gender_female,gender_male,gender_miss,registered_via_3,registered_via_4,registered_via_7,registered_via_9,registered_via_13,registered_via_16
0,XQxgAYj3klVKjR3oxPPXYYFp4soD4TuBghkhMTD4oTw=,0,1,3357,1134,1,0,0,0,0,...,0,0,0,1,0,0,1,0,0,0
1,UizsfmJb9mV54qE9hCYyU07Va97c0lCRLEQX3ae+ztM=,0,1,1949,1224,1,0,0,0,0,...,0,0,0,1,0,0,1,0,0,0
2,D8nEhsIOBSoE6VthTaqDX8U6lqjJ7dLdr72mOyLya2A=,0,1,1661,1204,1,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0
3,mCuD+tZ1hERA/o5GPqk38e041J8ZsBaLcu7nGoIIvhI=,0,1,1879,1878,1,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0
4,q4HRBfVSssAFS9iRfxWrohxuk9kCYMKjHOEagUMV6rQ=,0,1,1371,1233,1,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34398,Wwd/cudKVuLJ3txRVxlg2Zaeliu+LRUfiBmfrnxhRCY=,0,1,2543,1144,1,0,0,0,0,...,0,0,0,1,0,0,1,0,0,0
34399,g3JGnJX6Hg50lFbrNWfsHwCUmApIkiv2M8sXOaeXoIQ=,18,0,2196,1259,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
34400,IMaPMJuyN+ip9Vqi+z2XuXbFAP2kbHr+EvvCNkFfj+o=,0,1,2644,1146,1,0,0,0,0,...,0,0,0,1,0,0,1,0,0,0
34401,WAnCAJjUty9Stv8yKtV7ZC7PN+ilOy5FX3aIJgGPANM=,0,1,1835,1134,1,0,0,0,0,...,0,0,0,1,0,0,1,0,0,0


In [96]:
members.columns

Index(['msno', 'bd_missing', 'city', 'bd', 'gender', 'registered_via',
       'registration_init_time', 'expiration_date'],
      dtype='object')

In [90]:
members[['city', 'gender', 'registered_via']]

,city,gender,registered_via
0,1,NaN,7
1,1,NaN,7
2,1,NaN,4
3,1,NaN,9
4,1,NaN,4
...,...,...,...
34398,1,NaN,7
34399,4,male,3
34400,1,NaN,7
34401,1,NaN,7


In [46]:
X_train, y_train = get_features(train)

In [61]:
X_train

,source_system_tab,source_screen_name,source_type,city,bd,gender,registered_via,registration_init_time,expiration_date,song_length,genre_ids,artist_name,composer,lyricist,language
0,explore,Explore,online-playlist,1,NaN,NaN,7,20120102,20171005,206471.0,359,Bastille,Dan Smith| Mark Crew,NaN,52.0
1,my library,Local playlist more,local-playlist,13,24.0,female,9,20110525,20170911,284584.0,1259,Various Artists,NaN,NaN,52.0
2,my library,Local playlist more,local-playlist,13,24.0,female,9,20110525,20170911,225396.0,1259,Nas,N. Jones、W. Adams、J. Lordan、D. Ingle,NaN,52.0
3,my library,Local playlist more,local-playlist,13,24.0,female,9,20110525,20170911,255512.0,1019,Soundway,Kwadwo Donkoh,NaN,-1.0
4,explore,Explore,online-playlist,1,NaN,NaN,7,20120102,20171005,187802.0,1011,Brett Young,Brett Young| Kelly Archer| Justin Ebach,NaN,52.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7377413,my library,Local playlist more,local-playlist,4,26.0,female,9,20041218,20170930,267958.0,1259,Dr. Dre,Andre R. Young| Melvin Charles Bradford| Shawn...,NaN,52.0
7377414,search,Search,song,1,NaN,NaN,7,20130627,20171006,227404.0,465,R. City feat. Adam Levine,Theron Thomas| Timothy Thomas| Lukasz Gottwald...,Theron Thomas| Timothy Thomas| Lukasz Gottwald...,52.0
7377415,search,Search,song,1,NaN,NaN,7,20130627,20171006,258298.0,465,兄弟本色G.U.T.S. (姚中仁、張震嶽、頑童MJ116),姚中仁 MC HotDog 陳昱榕 Ｅ-SO 周文傑 KENZY 林睦淵,姚中仁 MC HotDog 陳昱榕 Ｅ-SO 周文傑 KENZY 林睦淵,3.0
7377416,discover,Discover Genre,online-playlist,5,NaN,NaN,9,20150213,20170918,524146.0,2122,Brad Mehldau Trio,NaN,NaN,52.0


# Задача 1:

Собрать более-менее приличную рекомендательную систему с помощью GBDT. Использование трюков из лекции (и не из неё), тюнинг и разумных объемов фича инжиниринг поощряется. 

In [59]:
xgb.sklearn.XGBClassifier().fit(X_train, y_train, eval_metric='auc')

ValueError: DataFrame.dtypes for data must be int, float or bool.
                Did not expect the data types in fields source_system_tab, source_screen_name, source_type, gender, genre_ids, artist_name, composer, lyricist

# Задача 2: 
Пользуясь информацией о треках построить неклассические эмбеддинги и попробовать использовать их:

1. показать симилары глазами
2. в качестве рекоммендера вместо GBDT (разницу в метриках) 
3. стек с GBDT (прирост / отсутствие / почему (советую поизучать SHAP values)